In [1]:
import numpy as np
import pandas as pd
import datetime
from astral.sun import sun
from astral import LocationInfo
import os

# Generate timestamps for instances with 5-minute intervals throughout the year
start_time = datetime.datetime(2022, 1, 1, 0, 0)
end_time = datetime.datetime(2022, 12, 31, 23, 59)
total_minutes = int((end_time - start_time).total_seconds() / 60)
minute_interval = 5  # Set the interval to 5 minutes

# Calculate the total number of instances
num_instances = total_minutes // minute_interval

timestamps = [start_time + datetime.timedelta(minutes=minute_interval * i) for i in range(num_instances)]

# Set the location for sunrise and sunset calculations
location = LocationInfo("St. Gallen", "Switzerland")
location.latitude = 47.4245
location.longitude = 9.3767

# Generate brightness values based on the time of day and the sunrise and sunset times
def generate_brightness(time):
    s = sun(location.observer, date=time.date())
    sunrise_hour = (s["sunrise"] - s["sunrise"].replace(hour=0, minute=0)).seconds / 3600
    sunset_hour = (s["sunset"] - s["sunset"].replace(hour=0, minute=0)).seconds / 3600
    current_hour = time.hour + time.minute / 60

    if sunrise_hour <= current_hour < (sunrise_hour + 6):
        return np.interp(current_hour, [sunrise_hour, sunrise_hour + 6], [0, 100])
    elif (sunset_hour - 6) <= current_hour < sunset_hour:
        return np.interp(current_hour, [sunset_hour - 6, sunset_hour], [100, 0])
    elif (sunrise_hour + 6) <= current_hour < (sunset_hour - 6):
        return 100
    else:
        return 0

brightness_data = [generate_brightness(t) for t in timestamps]

# Add noise to the brightness values
noise = np.random.normal(0, 5, len(brightness_data))
brightness_data = brightness_data + noise
brightness_data = np.clip(brightness_data, 0, 100)

# Store the data in a Pandas DataFrame
sensor_data = pd.DataFrame({
    'ts': timestamps,
    'br': brightness_data
})

# Save the data to a CSV file
sensor_data.to_csv(os.path.expanduser("~/Desktop/brightness_sensor_data_full_year.csv"), index=False)

In [2]:
import numpy as np
import pandas as pd
import datetime
from astral.sun import sun
from astral import LocationInfo
import os

# Generate timestamps for instances with 5-minute intervals throughout the year
start_time = datetime.datetime(2022, 1, 1, 0, 0)
end_time = datetime.datetime(2022, 12, 31, 23, 59)
total_minutes = int((end_time - start_time).total_seconds() / 60)
minute_interval = 5  # Set the interval to 5 minutes

# Calculate the total number of instances
num_instances = total_minutes // minute_interval

timestamps = [start_time + datetime.timedelta(minutes=minute_interval * i) for i in range(num_instances)]

# Set the location for sunrise and sunset calculations
location = LocationInfo("St. Gallen", "Switzerland")
location.latitude = 47.4245
location.longitude = 9.3767

def seasonality_factor(time):
    day_of_year = time.timetuple().tm_yday
    return 0.5 * (1 + np.cos((day_of_year - 172) / 365 * 2 * np.pi))

def daily_temperature_variation(time, min_temp, max_temp):
    current_hour = time.hour + time.minute / 60
    return min_temp + (max_temp - min_temp) * (1 + np.cos(2 * np.pi * (current_hour - 14) / 24)) / 2

def generate_temperature(time):
    winter_min_temp = 5
    winter_max_temp = 15
    summer_min_temp = 20
    summer_max_temp = 30

    min_temp = winter_min_temp + (summer_min_temp - winter_min_temp) * seasonality_factor(time)
    max_temp = winter_max_temp + (summer_max_temp - winter_max_temp) * seasonality_factor(time)

    return daily_temperature_variation(time, min_temp, max_temp)

temperature_data = [generate_temperature(t) for t in timestamps]

# Add noise to the temperature values
temperature_noise = np.random.normal(0, 1, len(temperature_data))
temperature_data = temperature_data + temperature_noise
temperature_data = np.clip(temperature_data, 5, 30)

# Store the data in a Pandas DataFrame
sensor_data = pd.DataFrame({
    'ts': timestamps,
    'temp': temperature_data
})

# Save the data to a CSV file
sensor_data.to_csv(os.path.expanduser("~/Desktop/temperature_sensor_data_full_year.csv"), index=False)